# **Loan Data Analysis using Lending Club Data**

## Part I: Obtaining the Data
---

In [117]:
import pandas as pd
import numpy as np
from tabulate import tabulate
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report,confusion_matrix

In [59]:
# Read in the CSV file 
DSLoanTrain = pd.read_csv('../data/loan-clean-version1.csv')
DSLoanTrain.head()

,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,...,revol_bal,revol_util,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,loan_status
0,1077501,5000,5000,4975.0,36 months,10.65,162.87,B,10+ years,RENT,...,13648,83.7,9,0,0,5861.071414,5831.78,5000.00,861.07,Fully Paid
1,1077430,2500,2500,2500.0,60 months,15.27,59.83,C,< 1 year,RENT,...,1687,9.4,4,0,0,1008.710000,1008.71,456.46,435.17,Charged Off
2,1077175,2400,2400,2400.0,36 months,15.96,84.33,C,10+ years,RENT,...,2956,98.5,10,0,0,3003.653644,3003.65,2400.00,603.65,Fully Paid
3,1076863,10000,10000,10000.0,36 months,13.49,339.31,C,10+ years,RENT,...,5598,21.0,37,0,0,12226.302210,12226.30,10000.00,2209.33,Fully Paid
4,1075269,5000,5000,5000.0,36 months,7.90,156.46,A,3 years,RENT,...,7963,28.3,12,0,0,5631.377753,5631.38,5000.00,631.38,Fully Paid


In [58]:
DSLoanTrain.dtypes

id                       int64
loan_amnt                int64
funded_amnt              int64
funded_amnt_inv        float64
term                    object
int_rate               float64
installment            float64
grade                   object
emp_length              object
home_ownership          object
annual_inc             float64
verification_status     object
purpose                 object
addr_state              object
dti                    float64
earliest_cr_line         int64
inq_last_6mths           int64
open_acc                 int64
pub_rec                  int64
revol_bal                int64
revol_util             float64
total_acc                int64
out_prncp                int64
out_prncp_inv            int64
total_pymnt            float64
total_pymnt_inv        float64
total_rec_prncp        float64
total_rec_int          float64
loan_status             object
dtype: object

In [71]:
DSLoanTrain[DSLoanTrain.isnull().any(axis=1)]

,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,...,revol_bal,revol_util,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,loan_status
3197,1016416,8000,8000,8000.0,36 months,19.42,294.95,E,3 years,RENT,...,0,NaN,10,0,0,10704.05102,10704.05,8000.00,2689.06,Fully Paid
4249,997734,6100,6100,6100.0,36 months,14.27,209.29,C,n/a,MORTGAGE,...,0,NaN,4,0,0,1755.22000,1755.22,989.91,471.90,Charged Off
4461,790093,7200,7200,7200.0,36 months,14.27,247.03,C,7 years,RENT,...,0,NaN,25,0,0,7306.07000,7306.07,7200.00,106.07,Fully Paid


In [100]:
class_mapping = {label:idx for idx, label in enumerate(np.unique(DSLoanTrain['term']))}
DSLoanTrain['term']=DSLoanTrain['term'].map(class_mapping)

class_mapping = {label:idx for idx, label in enumerate(np.unique(DSLoanTrain['grade']))}
DSLoanTrain['grade']=DSLoanTrain['grade'].map(class_mapping)

class_mapping = {label:idx for idx, label in enumerate(np.unique(DSLoanTrain['emp_length']))}
DSLoanTrain['emp_length']=DSLoanTrain['emp_length'].map(class_mapping)

class_mapping = {label:idx for idx, label in enumerate(np.unique(DSLoanTrain['home_ownership']))}
DSLoanTrain['home_ownership']=DSLoanTrain['home_ownership'].map(class_mapping)

class_mapping = {label:idx for idx, label in enumerate(np.unique(DSLoanTrain['verification_status']))}
DSLoanTrain['verification_status']=DSLoanTrain['verification_status'].map(class_mapping)

class_mapping = {label:idx for idx, label in enumerate(np.unique(DSLoanTrain['purpose']))}
DSLoanTrain['purpose']=DSLoanTrain['purpose'].map(class_mapping)

class_mapping = {label:idx for idx, label in enumerate(np.unique(DSLoanTrain['addr_state']))}
DSLoanTrain['addr_state']=DSLoanTrain['addr_state'].map(class_mapping)

class_mapping = {label:idx for idx, label in enumerate(np.unique(DSLoanTrain['loan_status']))}
DSLoanTrain['loan_status']=DSLoanTrain['loan_status'].map(class_mapping)

In [102]:
DSLoanTrain['emp_length'].value_counts()

1     2222
2      899
10     845
3      832
5      780
4      780
0      649
6      629
7      447
8      346
11     316
9      259
Name: emp_length, dtype: int64

In [101]:
DSLoanTrain['loan_status'].value_counts()

1    7487
0    1517
Name: loan_status, dtype: int64

In [103]:
# encoding loan_status data by 'find and replace'
#cleanup_nums = {"loan_status": {"Fully Paid":1, "Charged Off":0}}
#DSLoanTrain.replace(cleanup_nums, inplace=True)
#DSLoanTrain.head()

In [104]:
DSLoanTrain.head()

,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,...,revol_bal,revol_util,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,loan_status
0,1077501,5000,5000,4975.0,0,10.65,162.87,1,1,2,...,13648,83.7,9,0,0,5861.071414,5831.78,5000.00,861.07,1
1,1077430,2500,2500,2500.0,1,15.27,59.83,2,10,2,...,1687,9.4,4,0,0,1008.710000,1008.71,456.46,435.17,0
2,1077175,2400,2400,2400.0,0,15.96,84.33,2,1,2,...,2956,98.5,10,0,0,3003.653644,3003.65,2400.00,603.65,1
3,1076863,10000,10000,10000.0,0,13.49,339.31,2,1,2,...,5598,21.0,37,0,0,12226.302210,12226.30,10000.00,2209.33,1
4,1075269,5000,5000,5000.0,0,7.90,156.46,0,3,2,...,7963,28.3,12,0,0,5631.377753,5631.38,5000.00,631.38,1


In [105]:
DSLoanTrain = DSLoanTrain.select_dtypes(include=[np.number]).interpolate().dropna()
DSLoanTrain = DSLoanTrain.drop(["total_pymnt"], axis=1)
DSLoanTrain = DSLoanTrain.drop(["total_pymnt_inv"], axis=1)
DSLoanTrain = DSLoanTrain.drop(["total_rec_int"], axis=1)

In [107]:
#
yPredict = DSLoanTrain.loan_status
XClean = DSLoanTrain.drop(["loan_status"], axis=1)

In [108]:
#
X_train, X_test, y_train, y_test = train_test_split(XClean, yPredict, random_state=42, test_size=0.33)
#clf = RandomForestRegressor(n_jobs=2, n_estimators=1000)
#model = clf.fit(X_train, y_train)

In [113]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [116]:
#
mlp = MLPClassifier(hidden_layer_sizes=(30,30,30))
mlp.fit(X_train, y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(30, 30, 30), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [118]:
predictions = mlp.predict(X_test)
print(confusion_matrix(y_test, predictions))

[[ 491   18]
 [   4 2459]]


In [119]:
print(classification_report(y_test,predictions))

             precision    recall  f1-score   support

          0       0.99      0.96      0.98       509
          1       0.99      1.00      1.00      2463

avg / total       0.99      0.99      0.99      2972



In [125]:
#
X_train, X_test, y_train, y_test = train_test_split(XClean, yPredict, random_state=42, test_size=.33)
clf = RandomForestRegressor(n_jobs=2, n_estimators=1000)
model = clf.fit(X_train, y_train)

In [124]:
#
headers = ["name", "score"]
values = sorted(zip(X_train.columns, model.feature_importances_), key=lambda x:x[1] * -1)
print(tabulate(values, headers, tablefmt="plain"))

name                       score
total_rec_prncp      0.427501
funded_amnt          0.248147
loan_amnt            0.150895
funded_amnt_inv      0.122416
installment          0.0108364
id                   0.00548322
earliest_cr_line     0.00421792
term                 0.0041537
revol_util           0.00363214
annual_inc           0.00359457
dti                  0.00354887
total_acc            0.00285509
int_rate             0.00235712
revol_bal            0.00198491
open_acc             0.0017035
addr_state           0.00147387
emp_length           0.00128606
purpose              0.00100442
inq_last_6mths       0.000816795
verification_status  0.000706861
grade                0.000544486
home_ownership       0.000465566
pub_rec              0.000375509
out_prncp            0
out_prncp_inv        0
